# TEAM-SASA     
### Initial POC and EDA for Manila Bulletin Website

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from time import sleep

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 

## HELPER CLASS - WEBSCRAPING

In [2]:
DRIVER_PATH = 'C:\Program Files\chromedriver.exe'

class manilabulletinScraper:

    def __init__(self, search_url):
        self.urls = []
        self.search_url = search_url
        self.driver = webdriver.Chrome(DRIVER_PATH)

    def crawl(self):
        self.driver.get(self.search_url)
        totalpages = self.get_total_pages()
        self.get_pages(totalpages)

    def get_driver(self):
        return self.driver
        
    def get_urls(self):
        return self.urls

    def get_urls_count(self):
        return len(self.urls)

    def get_total_pages(self):
        pages = self.driver.find_element(By.CSS_SELECTOR, 'a.last')
        return int(pages.text)

    def add_urls(self, page):
        print(f'Start crawling article URLs for page {page}...')
        titles = self.driver.find_elements(By.XPATH, '//*[@id="news-feed"]/article/div/ul//h4/a')
        for title in titles:
            link = title.get_attribute('href')
            print(link)
            if (link != None) and (link not in self.urls):
                self.urls.append(link)
        print('done')

    def get_pages(self, totalpages):
        print(f'Total number of pages to crawl: {totalpages}', end='\n\n')
        # Page 1
        self.add_urls(1)
        # Page 2 onwards
        self.driver.maximize_window()
        for p in range(1, totalpages):
            x=0
            while x<1:
                print('before try')
                try:
                    main = WebDriverWait(self.driver, 10).until(
                                EC.presence_of_element_located((By.CSS_SELECTOR, 'i.mb-icon-arrow-right')))
                    self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight)', '') 
                    self.driver.find_element(By.CSS_SELECTOR, 'i.mb-icon-arrow-right').click()
                    sleep(5)
                    self.add_urls(p+1)
                    x +=1
                except:
                    self.close_popups()
                    print('exception')
            sleep(5)
            
    def close_popups(self):
        try:
            self.driver.find_element(By.XPATH, '//*[@id="survicate-box"]/div/div/div[3]/div[2]/button').click()
        except:
            pass
        try:
            self.driver.find_element(By.XPATH, '//*[@id="onesignal-slidedown-cancel-button"]').click()
        except:
            pass
        try:
            # Close footer ads
            self.driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[16]//following::div/p/span').click()
        except:
            pass
        try:
            self.driver.find_element(By.XPATH, '//*[@id="PopupSignupForm_0"]/div[2]/div[1]').click()
        except:
            pass
        try:
            self.driver.find_element(By.XPATH, '//*[@id="dismiss-button"]/div/span').click()
        except:
            pass
        try:
            self.driver.find_element(By.ID, 'dismiss-button').click()
        except:
            pass
        
        

    def scrape_data(self, url):
        ### PREPARE ARTICLE URL
        self.driver.get(url)
        sleep(15)

        ### PREPARE TITLE
        title = self.driver.find_element(By.CSS_SELECTOR, 'h2.title')
        title = title.text
        print(f'\n[Title]\n{title}')

        ### PREPARE PUBLISHED DATE
        pubdate = self.driver.find_element(By.CSS_SELECTOR, 'p.published')
        pubdate = pubdate.text
        print(f'\n[Date Pulished]\n{pubdate}')

        ### PREPARE AUTHOR NAME
        author = self.driver.find_element(By.CSS_SELECTOR, 'p.author')
        author = author.text
        print(f'\n[Author]\n{author}')

        # Try closing popups before clicking 'SEE THE RESULTS' button
        sleep(2)
        self.close_popups()

        ### PREPARE ARTICLE CONTENT
        article = self.driver.find_element(By.CSS_SELECTOR, 'section.article-content')
        article_content = article.text.replace('ADVERTISEMENT','').replace(' \n','').replace('\n ','').replace('\n\n','').replace('  ','') 
        print(f'\n[Content]\n{article_content}')

        ### PREPARE SENTIMENT RESULTS
        # Scroll to 'SEE THE RESULTS' button then simulate click to get sentiment results
        '''
        sleep(3)
        sentiment_classname = self.driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[16]/div[6]/div[4]/following::div').get_attribute('class')
        self.driver.execute_script(f'document.getElementsByClassName("{sentiment_classname}").scrollIntoView', '')
        
        # Automate click event to get sentiment results
        sentiment_index = 0
        for i in range(100):
            try:
                btn_sentiment = self.driver.find_element(By.XPATH, f'//*[@id="__next"]/div/div[16]/div[6]/div[4]/div[{i}]/div/div/span')
                if btn_sentiment.text.upper() == 'SEE THE RESULTS':
                    sleep(2)
                    self.close_popups()
                    action = ActionChains(self.driver)
                    action.move_to_element(btn_sentiment).click().perform()
                    # btn_sentiment.click()
                    sentiment_index = i
                    break
            except:
                pass

        sleep(3)
        # Clean some sentiment texts
        sentiment_results = self.driver.find_element(By.XPATH, f'//*[@id="__next"]/div/div[16]/div[6]/div[4]/div[{sentiment_index}]/div')
        sentiment_results = sentiment_results.text.replace('HOW OTHERS ARE FEELING', '').replace(' \n','').replace('\n ','').replace(' ARE ','-')
        # Convert each line of sentiment text to a list item
        sentiment_results = list(filter(None, sentiment_results.split('\n')))
        # Make a key-value pair of sentiment text and store it as dictionary object
        sentiment_results = {s.split('-')[1] : float(s.split('-')[0].split('%')[0])*.01 for s in sentiment_results}
        # Add a 0% value for sentiment key that does not have results in the current article and store as 'other sentiments'
        sentiment_keys = ['INSPIRED', 'HAPPY', 'AMUSED', 'NEUTRAL', 'ANNOYED', 'SAD', 'ANGRY', 'AFRAID']
        sentiment_others = {k : 0. for k in sentiment_keys if k not in sentiment_results}
        # Combine 'other sentiment dictionary' to the 'sentiment result dictionary' 
        sentiment_results = {**sentiment_results, **sentiment_others}
        print(f'\n[Sentiment]')
        print(sentiment_results)
        '''

        ### GET SENTIMENT RESULTS FROM ARTICLE
        return {'urls':url, 'titles':title, 'authors':author, 'published_dates':pubdate, 'articles_content':article_content}
    

# RUN WEB SCRAPING

In [3]:
KEYWORD = 'covid+vaccine'
SEARCH_URL = 'https://mb.com.ph/?s=Covid%2BVaccine'

scraper = manilabulletinScraper(SEARCH_URL)
scraper.crawl()

dataset = []
print(f'\nTotal number of articles: {scraper.get_urls_count()}')
for i,url in enumerate(scraper.get_urls()):
    try:
        print(f'\n\n\n[{i+1}]{" ":>5}SCRAPING DATA  ~>  {url}')
        data = scraper.scrape_data(url)
        dataset.append(data)
    except Exception as e:
        print(f'SCRAPING ERROR: {e}')
        pass

scraper.get_driver().quit()

Total number of pages to crawl: 56

Start crawling article URLs for page 1...
https://mb.com.ph/2021/03/05/first-outside-metro-covid-vaccines-reach-soldiers-in-cdo/
https://mb.com.ph/2021/03/05/dont-blame-us-pao-says-on-delay-in-arrival-of-covid-19-vaccines/
https://mb.com.ph/2021/03/05/astrazeneca-vaccine-can-give-protection-against-severe-covid-19-health-expert-says/
https://mb.com.ph/2021/03/05/malabon-city-starts-covid-19-vaccine-rollout/
https://mb.com.ph/2021/03/05/go-jab-govt-officials-with-covid-19-vaccines-after-essential-sectors/
https://mb.com.ph/2021/03/05/eu-commits-to-provide-affordable-fair-access-to-covid-19-vaccines-for-filipinos/
https://mb.com.ph/2021/03/05/new-evidence-shows-covid-19-antibodies-vaccines-less-effective-against-variants/
https://mb.com.ph/2021/03/04/laguna-receives-first-batch-of-covid-19-vaccines/
https://mb.com.ph/2021/03/04/ph-receives-millionth-covid-19-vaccine-shot-with-arrival-of-astrazeneca-doses/
https://mb.com.ph/2021/03/04/uk-to-fund-study-o

before try
Start crawling article URLs for page 6...
https://mb.com.ph/2021/02/26/us-fda-advisers-to-meet-on-evaluating-johnson-johnson-covid-19-vaccine/
https://mb.com.ph/2021/02/26/thailand-receives-1st-batch-of-covid-19-vaccines-from-chinas-sinovac/
https://mb.com.ph/2021/02/26/china-to-promote-equitable-global-distribution-of-covid-19-vaccines-foreign-ministry/
https://mb.com.ph/2021/02/25/govt-fast-tracking-covid-19-vaccines-acquisition-go/
https://mb.com.ph/2021/02/25/ph-offers-to-pay-more-as-a-technique-to-get-covid-vaccine-early/
https://mb.com.ph/2021/02/25/johnson-johnson-vaccine-highly-effective-against-severe-covid/
https://mb.com.ph/2021/02/25/large-real-world-study-confirms-pfizer-covid-vaccine-94-effective/
https://mb.com.ph/2021/02/25/doh-not-consulted-on-proposal-to-deploy-healthcare-workers-in-uk-germany-in-exchange-for-covid-19-vaccines/
https://mb.com.ph/2021/02/24/only-10-countries-cornered-initial-supplies-of-covid-19-vaccines-duterte/
https://mb.com.ph/2021/02/24

before try
Start crawling article URLs for page 11...
https://mb.com.ph/2021/02/13/moderna-wants-to-pack-50-more-covid-vaccine-per-vial/
https://mb.com.ph/2021/02/13/electric-coops-offer-sites-for-covid-19-vaccine-rollout/
https://mb.com.ph/2021/02/12/fda-hopes-indemnification-issues-on-covid-19-vaccines-will-be-settled-soon/
https://mb.com.ph/2021/02/12/nurses-ask-govt-to-be-transparent-accountable-in-its-covid-19-vaccine-roll-out/
https://mb.com.ph/2021/02/12/muntinlupa-intensifies-campaign-to-encourage-residents-to-get-covid-19-vaccine/
https://mb.com.ph/2021/02/12/us-acquires-200-mn-more-covid-vaccine-doses/
https://mb.com.ph/2021/02/12/hpaac-opposes-senate-bill-allowing-lgus-procurement-of-covid-19-vaccines/
https://mb.com.ph/2021/02/11/angara-assures-swift-approval-of-bill-exempting-covid-19-vaccine-purchases-from-stringent-procurement-rules/
https://mb.com.ph/2021/02/11/bulacan-province-prepares-for-upcoming-covid-19-vaccine/
https://mb.com.ph/2021/02/11/recto-dont-demonize-chin

before try
Start crawling article URLs for page 17...
https://mb.com.ph/2021/02/01/uk-trains-volunteer-vaccine-army-in-covid-inoculation-race/
https://mb.com.ph/2021/01/31/isko-follow-health-protocols-while-waiting-for-covid-19-vaccine/
https://mb.com.ph/2021/01/31/leni-bats-for-fda-approved-covid-19-vaccine/
https://mb.com.ph/2021/01/31/pentagon-pauses-plan-to-give-covid-vaccine-to-guantanamo-detainees/
https://mb.com.ph/2021/01/30/eu-adopts-covid-19-vaccine-export-control-scheme/
https://mb.com.ph/2021/01/30/hungary-first-in-eu-to-approve-chinese-covid-vaccine/
https://mb.com.ph/2021/01/29/jj-vaccine-provides-strong-shield-against-severe-covid-19/
https://mb.com.ph/2021/01/29/ph-negotiating-with-25-covid-19-vaccine-developers-dost/
https://mb.com.ph/2021/01/29/carmona-lgu-conducts-simulation-for-covid-19-vaccine-rollout/
https://mb.com.ph/2021/01/29/manila-lgu-unveils-covid-19-vaccine-storage-facility/
https://mb.com.ph/2021/01/28/drug-firm-to-hold-covid-vaccine-clinical-trial-in-bac

before try
Start crawling article URLs for page 23...
https://mb.com.ph/2021/01/15/use-of-different-brands-of-covid-19-vaccines-not-advisable-duque/
https://mb.com.ph/2021/01/15/pfizer-commits-to-give-ph-covid-vaccine-as-quickly-as-possible/
https://mb.com.ph/2021/01/15/imee-urges-govt-give-weight-to-covid-vaccines-with-asian-participants/
https://mb.com.ph/2021/01/15/lacson-rebuts-galvez-denies-politicizing-covid-vaccine-talks/
https://mb.com.ph/2021/01/15/sheryl-cruz-jeric-gonzales-and-sunshine-dizon-weigh-in-on-covid-19-vaccine/
https://mb.com.ph/2021/01/15/muntinlupa-to-buy-additional-200000-doses-of-covid-19-vaccine-from-astrazeneca/
https://mb.com.ph/2021/01/15/mobile-hybrid-containers-for-deep-freeze-transport-storage-and-distribution-of-covid-19-vaccines-in-the-philippines/
https://mb.com.ph/2021/01/15/davao-health-officials-undergo-training-to-administer-covid-19-vaccine/
https://mb.com.ph/2021/01/15/govt-yet-to-tap-cold-chain-industry-for-covid-19-vaccine-distribution/
https:

...


[Title]
WHO: COVID-19 is 10 times more deadly than swine flu; 70 vaccines in the works

[Date Pulished]
Published April 14, 2020, 12:00 AM

[Author]
by manilabulletin_admin

[Content]

By Agence France-Presse and Bloomberg
GENEVA – The novel coronavirus is 10 times more deadly than swine flu, which caused a global pandemic in 2009, the World Health Organization (WHO) said Monday, stressing a vaccine would be necessary to fully halt transmission.
The WHO said there are 70 coronavirus vaccines in development globally, with three candidates already being tested in human trials, as drugmakers race to find a cure for the deadly pathogen.
A logo is pictured on the headquarters of the World Health Orgnaization
(WHO) ahead of a meeting of the Emergency Committee on the novel coronavirus (2019-nCoV) in Geneva, Switzerland, January 30, 2020. (REUTERS / Denis Balibouse / FILE PHOTO / MANILA BULLETIN)The furthest along in the clinical process is an experimental vaccine developed by Hong Kong-list


[Title]
Duterte: We're doomed if there's no COVID-19 vaccine in 6 months

[Date Pulished]
Published April 4, 2020, 6:56 AM

[Author]
by manilabulletin_admin

[Content]

By Argyll Cyrus Geducos
President Duterte said the country is doomed if no vaccine against the coronavirus disease (COVID-19) is developed soon because the funds allocated for the pandemic crisis will not last long.
President Rodrigo Roa Duterte
(KING RODRIGUEZ/PRESIDENTIAL PHOTO / MANILA BULLETIN FILE PHOTO)
In his late-night public address Friday, President Duterte gave the assurance that the P275 billion budget to address the pandemic will not be pocketed by officials, but said it will run out fast.“Sabi ko may pera tayo. Hindi ba? O ngayon dumating, kailangan natin ng bilyon. Pero hindi ito tatagal. Dadating ito ng six months, patay na tayo,” he said.
(I said we have the money, right? Now it’s here, and we really need billions. But this will not last. It may last six months but beyond that, we’re doomed.)
“If they 


[Title]
Manufacturing firm on a rush to develop COVID-19 vaccine from tobacco

[Date Pulished]
Published March 25, 2020, 12:00 AM

[Author]
by manilabulletin_admin

[Content]

Medicago, a Canada-based company, announced that it now focuses on developing a plant-based vaccine against coronavirus disease which it hopes to submit for approval by the US Food and Drug Administration very soon.
Medicago said it successfully produced a Virus-Like Particle (VLP) of the coronavirus just 20 days after receiving the genetic sequence of severe acute respiratory syndrome coronavirus 2 (SARS-COV2), the virus responsible for COVID-19.
This is the breakthrough first step in developing a vaccine.Medicago already uses a plant-based technology to develop vaccines for seasonal flu, but is now working on developing a vaccine for COVID-19.
Instead of working with live virus in chicken eggs as a bioreactor, Medicago uses tobacco plants in what it describes as a faster and more efficient process of producing


[Title]
Duterte optimistic that Russia, US, Japan will develop COVID-19 vaccine ‘in a matter of days’

[Date Pulished]
Published March 10, 2020, 12:00 AM

[Author]
by manilabulletin_admin

[Content]

By Genalyn Kabiling
President Duterte is optimistic that Russia, the United States, and Japan could develop a vaccine against the new coronavirus disease “in a matter of days.”
President Rodrigo Roa Duterte
(TOTO LOZANO/PRESIDENTIAL PHOTO/ MANILA BULLETIN)
Amid the global outbreak of the coronavirus, the President said the best and brightest scientific minds are expected to work “overtime” for a potential vaccine.“With the great strides now of science, in a matter of days, I’m sure Russia — the brightest of the brightest — America, Japan, they are working overtime to come up with the vaccine,” Duterte said in an assembly of the country’s mayors in Pasay City.
“It takes time pero kaya na ng mga mundo. Kaya na ng mga gobyerno (but the world has the capability. The governments are capable), 


[Title]
PHAP to immediately bring new COVID-19 vaccines once available

[Date Pulished]
Published February 22, 2020, 12:00 AM

[Author]
by manilabulletin_admin

[Content]

By Bernie Cahiles-Magkilat
The Pharmaceutical Association of the Philippines (PHAP), which represents the research-based pharmaceutical and healthcare sector in the country, vowed to immediately bring new vaccines and technologies to cure and detect coronavirus disease-2019 (COVID-19) once these are available.
“Once available, our goal is to immediately bring these life-saving innovations into the country in partnership with the government,” said PHAP President Dr. Beaver Tamesis.Tamesis stressed, “Now more than ever, there is need for strong public-private collaboration to contain health threats.”
He added that the role of the pharmaceutical industry is crucial in efforts to stop the spread of COVID-19. Pharmaceutical companies enable the research and development of innovations and technologies for the detection of

# EDA

In [4]:
df = pd.DataFrame(dataset)
df

,urls,titles,authors,published_dates,articles_content
0,https://mb.com.ph/2021/03/05/first-outside-met...,First outside Metro: COVID vaccines reach sold...,by Martin Sadongdong,"Published March 5, 2021, 5:15 PM",\nThe Armed Forces of the Philippines (AFP) be...
1,https://mb.com.ph/2021/03/05/dont-blame-us-pao...,‘Don’t blame us’ – PAO says on delay in arriva...,by Jeffrey Damicog,"Published March 5, 2021, 2:26 PM",\n“Don’t blame us. We are just doing our job.”...
2,https://mb.com.ph/2021/03/05/astrazeneca-vacci...,AstraZeneca vaccine can give protection agains...,by Analou De Vera,"Published March 5, 2021, 12:47 PM",\nAstraZeneca’s coronavirus disease (COVID-19)...
3,https://mb.com.ph/2021/03/05/malabon-city-star...,Malabon City starts COVID-19 vaccine rollout,by Allysa Nievera,"Published March 5, 2021, 12:11 PM",\nThe local government of Malabon City launche...
4,https://mb.com.ph/2021/03/05/go-jab-govt-offic...,Go: Jab gov’t officials with COVID-19 vaccines...,by Mario Casayuran,"Published March 5, 2021, 10:53 AM","\nSenator Christopher “Bong” Go on Friday, Mar..."
...,...,...,...,...,...
877,https://mb.com.ph/2020/03/07/virus-folding-puz...,Virus-folding puzzle game might help find vacc...,by manilabulletin_admin,"Published March 7, 2020, 12:00 AM",\nBy Rizal Obanil\nThis development has the ma...
878,https://mb.com.ph/2020/02/25/needed-vaccine-in...,Needed: vaccine & info on how Covid-1 spreads,by manilabulletin_admin,"Published February 25, 2020, 12:00 AM","The search for a vaccine for the coronavirus, ..."
879,https://mb.com.ph/2020/02/22/phap-to-immediate...,PHAP to immediately bring new COVID-19 vaccine...,by manilabulletin_admin,"Published February 22, 2020, 12:00 AM",\nBy Bernie Cahiles-Magkilat\nThe Pharmaceutic...
880,https://mb.com.ph/2020/02/14/as-scientists-rac...,As scientists race to produce Covid-19 vaccine...,by manilabulletin_admin,"Published February 14, 2020, 12:00 AM",\nBy Analou De Vera\nAn official of the World ...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 882 entries, 0 to 881
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   urls              882 non-null    object
 1   titles            882 non-null    object
 2   authors           882 non-null    object
 3   published_dates   882 non-null    object
 4   articles_content  882 non-null    object
dtypes: object(5)
memory usage: 34.6+ KB


In [6]:
df.describe().T

,count,unique,top,freq
urls,882,882,https://mb.com.ph/2021/02/10/no-delay-with-cov...,1
titles,882,882,Gov’t exploring multiple sources of COVID-19 v...,1
authors,882,94,by Agence France-Presse,128
published_dates,882,873,"Published April 25, 2020, 12:00 AM",3
articles_content,882,877,\nBy Agence France-Presse\nThe 193 members of ...,2


In [7]:
df.isna().sum()

urls                0
titles              0
authors             0
published_dates     0
articles_content    0
dtype: int64

In [8]:
### Save a CSV dataset
df.to_csv(f'rappler_{KEYWORD}.csv', index=False)

# WORDCLOUD

In [ ]:
from wordcloud import WordCloud
from PIL import Image
import numpy as np

In [ ]:
def grey_color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    return "hsl(0, 0%%, %d%%)" % np.random.randint(60, 100)

mask = np.array(Image.open('bg.png'))

words = ' '.join(df['articles_content'].tolist())
wordcloud = WordCloud(background_color='black', max_words=2000, width=1024, height=720, mask=mask)
wordcloud.generate(words)

plt.figure(figsize=(20, 8), dpi=300)
plt.title('Rappler on COVID Vaccine in PH (Word Cloud)')
plt.imshow(wordcloud.recolor(color_func=grey_color_func, random_state=42), interpolation='bilinear')
plt.axis('off')
plt.show()